# Differentiable Neural Computer

![DNC architecture](https://storage.googleapis.com/deepmind-live-cms/images/dnc_figure1.width-1500_Zfxk87k.png "DNC architecture")

![DNC model](images/dnc_model.png "DNC model")


Original paper:
[Graves, Alex, et al. "Hybrid computing using a neural network with dynamic external memory." Nature 538.7626 (2016): 471-476.](https://www.nature.com/nature/journal/v538/n7626/abs/nature20101.html)

Author blog post:
[Differentiable neural computers](https://deepmind.com/blog/differentiable-neural-computers/)

Author implementation (Tensorflow + Sonnet):
[deepmind/dnc](https://github.com/deepmind/dnc)

## Definitions

$t \in \mathbb{N}$: time step

$X$: input domain

$Y$: output domain

$[v_1; \ldots; v_k]$: concatenation of $k$ vectors

$\circ$: elementwise multiplication

$\mathcal{S}_K$: $(K-1)$-dimensional unit simplex;
$\mathcal{S}_K = \{\alpha \in \mathbb{R}^K: \alpha_i \in [0, 1], \sum_{i=1}^K \alpha_i = 1\}$

$\Delta_K$: non-negative orthant of $\mathbb{R}^K$ with $\mathcal{S}_K$ as boundary;
$\Delta_K = \{\alpha \in \mathbb{R}^K: \alpha_i \in [0, 1], \sum_{i=1}^K \alpha_i \leq 1\}$

### Activations

* $\sigma(x) = (1 + e^{-x})^{-1}$
* $oneplus(x) = 1 + \log(1 + e^x)$
* $softmax(x)[i] = e^{x[i]} (\sum_{j=1}^n e^{x[j]})^{-1}$

### Hyperparameters

$N \in \mathbb{N}$: number of memory rows or locations

$W \in \mathbb{N}$: number of memory columns or word length

$R \in \mathbb{N}$: number of read heads

$L \in \mathbb{N}$: number of controller network layers

### Inputs

$M_t \in \mathbb{R}^{N \times W}$: memory matrix at time $t$

$x_t \in \mathbb{R}^X$: input vector at time $t$

$z_t \in \mathbb{R}^Y$: target vector at time $t$

$r_t^i \in \mathbb{R}^W$: read vector at time $t$ from read head $i$

### Controller network

$\mathcal{N}$: controller network

$\mathcal{X}_t \in \mathbb{R}^{X + RW}$: controller input vector at time $t$; $\mathcal{X}_t = [x_t; r_{t-1}^1; \ldots, r_{t-1}^R]$

$\theta$: controller parameter vector (all the weights)

* Recurrent Neural Network: $(\nu_t, \xi_t) = \mathcal{N}([\mathcal{X}_1; \ldots; \mathcal{X}_t]; \theta)$

* Feed-forward Neural Network: $(\nu_t, \xi_t) = \mathcal{N}(\mathcal{X}_t; \theta)$

Deep LSTM variant used in the paper:

$$i_t^l = \sigma(W_i^l[\mathcal{X}_t; h_{t - 1}^l; h_t^{l-1}] + b_i^l)$$

$$f_t^l = \sigma(W_f^l[\mathcal{X}_t; h_{t - 1}^l; h_t^{l-1}] + b_f^l)$$

$$s_t^l = f_t^l s_{t-1}^l + i_t^l tanh(W_s^l[\mathcal{X}_t; h_{t - 1}^l; h_t^{l-1}] + b_s^l)$$

$$o_t^l = \sigma(W_o^l[\mathcal{X}_t; h_{t - 1}^l; h_t^{l-1}] + b_o^l)$$

$$h_t^l = o_t^l tanh(s_t^l)$$

Where for layer $l$ at time $t$:

$h_t^l$: hidden state

$i_t^l$: input gate

$f_t^l$: forget gate

$s_t^l$: state gate

$o_t^l$: output gate

$h_t^0 = 0$, $h_0^l = 0$ and $s_0^l = 0$.

### Network Weights / Parameters

$W_{\xi}$: hidden to interface weight matrix

$W_y$: hidden to output weight matrix

$W_r \in \mathbb{R}^{RW \times Y}$: read vectors to output weight matrix

(also the hidden weights of the chosen architecture)

### Output

$y_t \in \mathbb{R}^Y$: output vector at time $t$;
$y_t = W_y[h_t^1; \ldots, h_t^L] + W_r[r_t^1; \ldots, r_t^R]$

$\xi_t \in \mathbb{R}^{(W \times R) + 3W + 5R + 3}$:
interface vector (interactions with the memory) at time $t$;
$\xi = W_{\xi}[h_t^1; \ldots, h_t^L]$

## Interface parameters

$\xi_t= [
k_t^{r,1}; \ldots; k_t^{r,R};
\hat{\beta}_t^{r,1}; \ldots; \hat{\beta}_t^{r,R};
k_t^w; \hat{\beta}_t^w; \hat{e}_t; v_t;
\hat{f}_t^1; \ldots; \hat{f}_t^R;
\hat{g}_t^a; \hat{g}_t^w;
\hat{\pi}_t^1; \ldots; \hat{\pi}_t^R
]$

$\{k_t^{r,i} \in \mathbb{R}^W; 1 \leq i \leq R\}$: read keys

$\{\beta_t^{r,i} = oneplus(\hat{\beta}_t^{r,i}) \in [1, \infty); 1 \leq i \leq R\}$: read strengths

$k_t^w \in \mathbb{R}^W$: write key

$\beta_t^w = oneplus(\hat{\beta}_t^w) \in [1, \infty)$: write strength

$e_t = \sigma(\hat{e}_t) \in [0, 1]^W$: erase vector

$v_t \in \mathbb{R}^W$: write vector

$\{f_t^i = \sigma(\hat{f}_t^i) \in [0, 1]; 1 \leq i \leq R\}$: free gates

$g_t^a = \sigma(\hat{g}_t^a) \in [0, 1]$: allocation gate

$g_t^w = \sigma(\hat{g}_t^w) \in [0, 1]$: write gate

$\{\pi_t^i = softmax(\hat{\pi}_t^i) \in \mathcal{S}_3; 1 \leq i \leq R\}$: read modes

## Read modes / Memory addressing / Content attention mechanisms

### Similarity meassure (cossine similarity)

Used for reading and writing and it is related to assosiative structures.

$$\mathcal{C}(M, k, \beta)[i] = \frac{\exp\{\mathcal{D(k, M[i, \cdot])\beta}\}}
{(\sum_{j=i}^N \exp\{\mathcal{D(k, M[j, \cdot])\beta}\})}$$

Where:

$\mathcal{C}(M, k, \beta) \in \mathcal{S}_N$ defines a normalized probability distribution over the memory locations,

$k \in \mathbb{R}^W$ is the lookup key,

$\beta \in [1, \infty)$ is the key strength,

and $\mathcal{D}$ is the cosine similarity defined as:

$$\mathcal{D}(u, v) = \frac{u \cdot v}{|u||v|}$$

### Usage vector and memory allocation

$\{f_t^i, 1 \leq i \leq R\}$: free gates; determine whether the most recently read locations can be freed.

$\psi_t \in [0, 1]^N$: retention vector; represents by how much each location will not be freed by the free gates;
$\psi_t = \Pi_{i=1}^R (1- f_t^i w_{t-1}^{r, i})$

$u_t \in [0, 1]^N$: usage vector; $u_t = (u_{t-1} + w_{t-1}^w - u_{t-1} \circ w_{t-1}^w) \circ \psi_t$ where $u_0 = 0$

A location $i$ is used if it has been retained by the free gates ($\psi_t[i] \approx 1$), and were either already in use or have just been written to.

$\phi_t \in \mathbb{Z}^N$: free list; is defined by sorting the indices of the memory locations in ascending order of $u_t$

$a_t \in \Delta_N$: allocation weighting; provides new locations for writing:

$$a_t[\phi_t[j]] = (1 - u_t[\phi_t[j]]) \Pi_{i=1}^{j-1} u_t[\phi_t[i]]$$

If all usages are 1, then a $a_t = 0$ and the controller can no longer allocate memory without first freeing used locations.

The sort operation induces discontinuities at the points at which the sort order changes. The authors ignore these discontinuities when calculating the gradient, as they do not seem to be relevant to learning.

### Temporal link matrix

$p_t \in \Delta_N$: precedence weighting;
$p_t = \left(1 - \sum_i w_t^w[i]\right) p_{t - 1} + w_t^w$ where $p_0 = 0$

$p_t[i]$ represents the degree to which location $i$ was the last one written to.

$L \in [0, 1]^{N \times N}$: temporal link matrix; where:

$$L_0[i, j] = 0, 1 \leq i, j \leq N$$

$$L_t[i, i] = 0, 1 \leq i \leq N$$

$$L_t[i, j] = (1 - w_t^w[i] - w_t^w[j]) L_{t - 1}[i, j] + w_t^w[i] p_{t - 1}[j]$$

$L[i, \cdot] \in \Delta_N$ and $L[\cdot, j] \in \Delta_N$ for all $1 \leq i, j \leq N$.

If $L[i, j] \approx 1$ then $i$ was written after $j$, otherwise $L[i, j] \approx 0$.

$Lw$ smoothly shifts the focus forwards to the locations written after those emphasized in $w$,
whereas $L^{\top}w$ shifts the focus backwards.

There is a sparse version of this matrix in the parper.

## Read and write weightings

### Calculate next state of memory (erase and write)

$c_t^w \in \mathcal{S}_N$: write content weighting;
$c_t^w = \mathcal{C}(M_{t - 1}, k_t^w, \beta_t^w)$

$w_t^w \in \Delta_N$: write weighting;
$w_t^w = g_t^w[g_t^a a_t + (1 - g_t^a) c_t^w]$

$M_t = M_{t - 1} \circ (E - w_t^w e_t^{\top}) + w_t^w v_t^{\top}$

Where $E$ is an $N \times W$ matrix of ones.

### Calculate read vectors

For each read head $i$ with $1 \leq i \leq R$:

$c_t^{r, i} \in \mathcal{S}_N$: read content weighting;
$c_t^{r, i} = \mathcal{C}(M_{t - 1}, k_t^{r, i}, \beta_t^{r, i})$

$f_t^i \in \Delta_N$: forward weighting; $f_t^i = L_t w_{t - 1}^{r, i}$

$b_t^i \in \Delta_N$: backward weighting; $b_t^i = L_t^{\top} w_{t - 1}^{r, i}$

$w^{r, i} \in \Delta_N$: read weighting;
$w^{r, i} = \pi_t^i[1] b_t^i + \pi_t^i[2] c_t^{r, i} + \pi_t^i[3] f_t^i$

$r_t^i$: read vector; $r_t^i = M_t^{\top} w_t^{r, i}$

If $\pi_t^i[2]$ dominates the read mode, then the weighting reverts to content lookup using $k_t^{r, i}$.

If $\pi_t^i[3]$ dominates, then the read head iterates through memory locations in the order they were written, ignoring the read key.

If $\pi_t^i[1]$ dominates, then the read head iterates in the reverse order.